In [1]:
# install the OpenJDK 11 JDK on a Debian or Ubuntu-based system in a quiet and non-interactive manner
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
# download a file from a website
!wget -nc -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
# extract the contents of a file
!tar xf spark-3.3.2-bin-hadoop3.tgz

In [2]:
 # API for interacting with the Spark
!pip install pyspark==3.3.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 17.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=bce36cfd128f15e55488c2852f5bfb51a9a2d8986e2c1a981f98aba234192d10
  Stored in directory: /root/.cache/pip/wheels/89/d6/52/1178e354ba2207673484f0ccd7b2ded0ab6671ae5c1fc5b49a
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [12]:
# explore files and directories of any locations in colab
!ls /content/spark-3.3.2-bin-hadoop3/

bin   data	jars	    LICENSE   NOTICE  R		 RELEASE  yarn
conf  examples	kubernetes  licenses  python  README.md  sbin


In [4]:
!ls /content/spark-3.3.2-bin-hadoop3/data

graphx	mllib  streaming


In [5]:
import os, sys
# set the environment variable to the locations where Spark and Java are installed.
os.environ["JAVA_HOME"] =  "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"
# add PySpark libraries to the system path:
sys.path.append("/content/spark-3.3.2-bin-hadoop3/python")
sys.path.append("/content/spark-3.3.2-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip")

In [13]:
# import dependencies
import pyspark.sql.types as T
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.ml.feature import Binarizer, Bucketizer, OneHotEncoder, VectorAssembler, StringIndexer, MinMaxScalerModel, \
StandardScaler, Imputer, Tokenizer,StopWordsRemover, MinMaxScaler, PolynomialExpansion
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator, ClusteringEvaluator, RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.mllib.evaluation import RankingMetrics
from random import randint
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np; import pandas as pd
from pyspark.sql.functions import concat, lit

from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
path = "/content/drive/MyDrive/Project Bigdata/DATA/"

# Run Local

In [8]:
# run locally
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark import SparkConf

spark = (SparkSession.builder.appName("movies (local)")
          .config("spark.master", "local[*]")
          # .remote('')
          .config("spark.hadoop.dfs.client.use.datanode.hostname", "true")
          .getOrCreate())

In [11]:
# spark.read.text('gutenberg_books/*.txt') # create folder and upload files if you cannot access hadoop
# to run the code below, you must change the hostname in colab at /etc/hosts
results = spark.read.option("header",True).csv('hdfs://sn1.bigdata.com:9000/movies/movies.csv')
  

results.show(10, truncate=False)
# results.orderBy(F.col('count').desc()).show(10)

+-----------------+---------------------------+----+
|actor            |title                      |year|
+-----------------+---------------------------+----+
|McClure, Marc (I)|Freaky Friday              |2003|
|McClure, Marc (I)|Coach Carter               |2005|
|McClure, Marc (I)|Superman II                |1980|
|McClure, Marc (I)|Apollo 13                  |1995|
|McClure, Marc (I)|Superman                   |1978|
|McClure, Marc (I)|Back to the Future         |1985|
|McClure, Marc (I)|Back to the Future Part III|1990|
|Cooper, Chris (I)|Me, Myself & Irene         |2000|
|Cooper, Chris (I)|October Sky                |1999|
|Cooper, Chris (I)|Capote                     |2005|
+-----------------+---------------------------+----+
only showing top 10 rows



In [12]:
spark.stop()

# Run Cluster Mode

In [27]:
spark.stop()

In [17]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark import SparkConf

spark = SparkSession.builder.appName("model (spark cluster)")\
            .config("spark.master", "spark://mn1.bigdata.com:7077")\
            .config("spark.hadoop.dfs.client.use.datanode.hostname", "true").getOrCreate()
spark.sparkContext.setLogLevel('INFO')
spark

Py4JJavaError: ignored

In [ ]:
spark.conf

In [10]:
results = spark.read.option("header",True).csv('movies.csv')

NameError: ignored

In [45]:
data = [("John Doe", "johndoe@example.com", "123 Main St.", "New York"),
        ("Jane Smith", "janesmith@example.com", "456 Elm St.", "Chicago")]
df = spark.createDataFrame(data, ["name", "email", "address", "city"])
df.show()        

Py4JJavaError: ignored

In [14]:
results = spark.read.option("header",True).csv('/home/spark/movies.csv')
result.show()

NameError: ignored

In [18]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.ml import Pipeline
from pyspark.ml.feature import Binarizer, Bucketizer, OneHotEncoder, VectorAssembler, StringIndexer, MinMaxScalerModel, \
StandardScaler, Imputer, Tokenizer,StopWordsRemover, MinMaxScaler, PolynomialExpansion
import pyspark.ml.feature as MF
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

if __name__ == '__main__':
  spark = SparkSession.builder.appName("model (spark cluster)")\
              .config("spark.master", "spark://mn1.bigdata.com:7077")\
              .config("spark.hadoop.dfs.client.use.datanode.hostname", "true").getOrCreate()
  spark.sparkContext.setLogLevel('INFO')

  results = spark.read.option("header",True).csv('/home/spark/movies.csv')

  results.show()

Py4JJavaError: ignored

In [14]:
results = spark.read.option("header",True).csv('hdfs://sn1.bigdata.com:9000/movies/movies.csv')
  

# results.show(10, truncate=False)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [15]:
spark.stop()

In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
results.show(2)

In [ ]:
spark.stop()

# Run Model

In [ ]:
# run locally
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark import SparkConf

spark = (SparkSession.builder.appName("movies (local)")
          .config("spark.master", "local[*]")
          # .remote('')
          .config("spark.hadoop.dfs.client.use.datanode.hostname", "true")
          .getOrCreate())

### Load Data

In [ ]:
features = np.load(path + 'image_array18000_86.npy')[::100]
names = np.load(path + 'image_name.npy')[::100]

In [ ]:
features.shape

(180, 2048)

In [ ]:
# with open(path+'image_array_86.npy', 'wb') as f:
#   np.save(f, features)

# with open(path+'image_name_86.npy', 'wb') as f:
#   np.save(f, names)

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType
from pyspark.ml.linalg import VectorUDT
from pyspark.ml.linalg import Vectors


names = [int(x) for x in names]
features = [Vectors.dense(features[i]) for i in range(len(features))]
schema = StructType([StructField("names", IntegerType()),StructField("features", VectorUDT(), True)])
 
data = [(names[i], features[i]) for i in range(len(names))]
df2 = spark.createDataFrame(data, schema)
df2 = df2.withColumn('names', concat(lit('0'), df2['names']))
df2.show()

TypeError: ignored

In [ ]:
chema


StructType([StructField('names', IntegerType(), True), StructField('features', VectorUDT(), True)])